In [ ]:
import os
import time
import datetime
import numpy as np
import pandas as pd
import ccxt
from dotenv import load_dotenv

# 載入環境變數
load_dotenv()

# 設定Binance API秘鑰和秘鑰
api_key = os.getenv('BINANCE_API_KEY')
secret_key = os.getenv('BINANCE_SECRET_KEY')

# 初始化Binance交易所
exchange = ccxt.binance({
    'apiKey': api_key,
    'secret': secret_key,
    'enableRateLimit': True
})

# 獲取當前帳戶資訊
def get_account():
    account = exchange.fetch_balance()
    return account

# 獲取指定交易對的K線資料
def get_historical_data(symbol, timeframe, limit):
    data = exchange.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

# 計算n天區間的最高價和最低價
def calculate_donchian_channel(df, n):
    df['Upper'] = df['high'].rolling(n).max()
    df['Lower'] = df['low'].rolling(n).min()
    return df

# 計算區間突破信號
def calculate_signal(df):
    df['Signal'] = np.where(df['close'] > df['Upper'].shift(1), 1, np.where(df['close'] < df['Lower'].shift(1), -1, 0))
    return df

# 計算持倉信號
def calculate_position(df):
    df['Position'] = df['Signal'].shift(1)
    return df

# 下單交易
def execute_trade(symbol, side, quantity):
    try:
        order = exchange.create_market_order(symbol=symbol, type='market', side=side, amount=quantity)
        print('Order executed:', order)
    except Exception as e:
        print('Order failed:', e)

# 主程式
if __name__ == '__main__':
    # 設定交易對、時間週期和區間長度
    symbol = 'BTC/USDT'
    timeframe = '1d'
    n = 20

    while True:
        # 獲取K線資料，並計算區間和信號
        data = get_historical_data(symbol=symbol, timeframe=timeframe, limit=n+1)
        data = calculate_donchian_channel(data, n)
        data = calculate_signal(data)
        data = calculate_position(data)

        # 獲取當前帳號資訊
        account = get_account()
        usdt_balance = account['free']['USDT']

        # 如果持倉為0且當前信號為1，則做多
        if data['Position'].iloc[-1] == 1 and usdt_balance > 10:
            quantity = round((usdt_balance / data['close'].iloc[-1]) * 0.95, 6)
            execute_trade(symbol=symbol, side='buy', quantity=quantity)
            print('Buy', quantity, 'at', data['close'].iloc[-1])

        # 如果持倉為多且當前信號為-1，則清倉
        elif data['Position'].iloc[-1] == -1 and us